In [1]:
#Add Python module called Mediapipe
!pip install mediapipe

     --------------------------------------- 49.8/49.8 MB 13.9 MB/s eta 0:00:00
  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-win_amd64.whl (42.5 MB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.0 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.12.6 which is incompatible.


In [1]:
# Import the much needed stuff for training
import pandas as pd
import numpy as np
import tensorflow as tf
import mediapipe as mp
import os
import csv
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.utils import to_categorical

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

# Checking Tensorflow Version
tf.__version__

'2.9.0'

In [2]:
# Read CSV file for Training the model using Pandas
df_train = pd.read_csv("totol_1output_copy.csv", header=0)

# First we must sort the values of the dataset according to the Alphabets
df_train = df_train.sort_values(by=["class_type"])

df_train

,class_type,thumb_CmcX,thumb_CmcY,thumb_CmcZ,thumb_McpX,thumb_McpY,thumb_McpZ,thumb_IpX,thumb_IpY,thumb_IpZ,...,pinky_McpZ,pinky_PipX,pinky_PipY,pinky_PipZ,pinky_DipX,pinky_DipY,pinky_DipZ,pinky_TipX,pinky_TipY,pinky_TipZ
0,0,0.012261,-0.054713,-0.064790,0.046395,-0.081590,-0.106398,0.074692,-0.068785,-0.142921,...,-0.064073,0.047597,0.140222,-0.112481,0.002290,0.150317,-0.113498,-0.044463,0.142457,-0.103716
23549,0,0.025092,-0.064440,-0.139927,0.098135,-0.107076,-0.221401,0.196540,-0.083356,-0.286363,...,-0.070091,0.295098,0.167263,-0.158358,0.201370,0.160428,-0.153877,0.148551,0.124910,-0.125677
23550,0,0.022196,-0.065882,-0.132180,0.102703,-0.110805,-0.203751,0.205004,-0.087554,-0.259828,...,-0.049213,0.308012,0.154426,-0.132990,0.220689,0.154105,-0.128571,0.168385,0.121921,-0.100112
23551,0,0.029679,-0.073904,-0.117726,0.113281,-0.122347,-0.179320,0.214193,-0.096491,-0.227835,...,-0.043005,0.313024,0.145701,-0.125747,0.223044,0.143749,-0.123801,0.169800,0.109868,-0.096811
23552,0,0.035407,-0.080681,-0.098662,0.125531,-0.126880,-0.148170,0.230018,-0.100868,-0.189195,...,-0.032644,0.318775,0.128685,-0.110378,0.231798,0.126545,-0.107725,0.173329,0.097503,-0.080556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37095,4,0.039656,-0.033751,-0.127444,0.107512,-0.046348,-0.189851,0.195785,-0.005595,-0.231522,...,-0.025782,0.275776,0.102451,-0.089464,0.218746,0.112286,-0.098906,0.174507,0.092683,-0.089300
37094,4,0.045146,-0.038217,-0.120844,0.112096,-0.053228,-0.181130,0.200864,-0.010681,-0.223143,...,-0.033020,0.258502,0.115805,-0.097260,0.202204,0.115296,-0.104968,0.157365,0.088228,-0.092823
37093,4,0.045179,-0.046358,-0.112949,0.113749,-0.068758,-0.171630,0.201301,-0.025802,-0.214236,...,-0.042111,0.252671,0.116860,-0.104021,0.192834,0.110457,-0.107463,0.153796,0.079014,-0.091955
37065,4,0.048017,-0.042993,-0.113411,0.125094,-0.058465,-0.168289,0.216048,-0.007725,-0.206201,...,-0.031464,0.253758,0.127802,-0.094675,0.186126,0.119191,-0.094628,0.151503,0.080770,-0.076026


In [3]:
# Read CSV file for Validation or Testing the Model using Pandas
df_test = pd.read_csv("output0_1_copy.csv", header=0)

# First we must sort the values of the dataset according to the Alphabets
df_test = df_test.sort_values(by=["class_type"])

df_test

,class_type,thumb_CmcX,thumb_CmcY,thumb_CmcZ,thumb_McpX,thumb_McpY,thumb_McpZ,thumb_IpX,thumb_IpY,thumb_IpZ,...,pinky_McpZ,pinky_PipX,pinky_PipY,pinky_PipZ,pinky_DipX,pinky_DipY,pinky_DipZ,pinky_TipX,pinky_TipY,pinky_TipZ
0,2,0.078050,0.052410,-0.097754,0.175706,0.077343,-0.128307,0.275970,0.113071,-0.136625,...,0.054411,0.357743,0.057897,0.018375,0.358115,0.126840,-0.009358,0.339377,0.179944,-0.019861
203,2,-0.012620,-0.044744,-0.123135,0.018767,-0.079958,-0.195871,0.078375,-0.064984,-0.246966,...,-0.099393,0.244468,0.119232,-0.166323,0.178340,0.133267,-0.168216,0.127710,0.125424,-0.154277
202,2,-0.008097,-0.045152,-0.121893,0.024023,-0.080934,-0.193070,0.084990,-0.066602,-0.242528,...,-0.089771,0.246310,0.127228,-0.152808,0.181133,0.137645,-0.155199,0.134478,0.129427,-0.141829
201,2,-0.009345,-0.047620,-0.120329,0.021717,-0.085917,-0.190250,0.081786,-0.071348,-0.238580,...,-0.090403,0.238301,0.129838,-0.149954,0.173444,0.136199,-0.149373,0.132625,0.123766,-0.134313
200,2,-0.007643,-0.046334,-0.114762,0.018584,-0.086358,-0.183096,0.076390,-0.072655,-0.231085,...,-0.084937,0.243161,0.126474,-0.139383,0.179160,0.133866,-0.137239,0.135621,0.121028,-0.121733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,4,0.052386,-0.033384,-0.098482,0.138610,-0.056004,-0.137836,0.236382,-0.034437,-0.167124,...,0.010491,0.280510,0.069187,-0.040047,0.219858,0.082490,-0.040654,0.181638,0.064963,-0.023303
1989,4,0.048063,-0.032764,-0.101829,0.137323,-0.053630,-0.141715,0.239965,-0.031687,-0.169697,...,0.017140,0.305112,0.069570,-0.035449,0.245721,0.090840,-0.039726,0.203419,0.077289,-0.025330
1988,4,0.037981,-0.035757,-0.097611,0.122475,-0.065621,-0.141231,0.224929,-0.046344,-0.173946,...,-0.014423,0.309466,0.082511,-0.071618,0.237189,0.098081,-0.065985,0.197453,0.075824,-0.043239
1997,4,0.053431,-0.017564,-0.097405,0.140951,-0.023345,-0.126577,0.241364,0.000656,-0.140527,...,0.058359,0.293261,0.026037,0.026640,0.243700,0.051435,0.023560,0.205439,0.047243,0.034067


In [4]:
# Put Categorical using Pandas
df_train["class_type"] = pd.Categorical(df_train["class_type"])
df_train["class_type"] = df_train.class_type.cat.codes

df_test["class_type"] = pd.Categorical(df_test["class_type"])
df_test["class_type"] = df_test.class_type.cat.codes

In [5]:
# Copy Label and Feature for training
y_train = df_train.pop("class_type")
x_train = df_train.copy()

y_test = df_test.pop("class_type")
x_test = df_test.copy()

# Copied Features turn to Array by using NumPy
x_train = np.array(x_train)
x_test = np.array(x_test)

In [6]:
# Check Array Shape before transformation
print(x_train.shape)
print(x_test.shape)

# Since the array shape is 1x1, we must turn it into 1x10x1 so we can feed it into the model
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Check Array Shape after transformation
print(x_train.shape)
print(x_test.shape)

(67804, 60)
(2988, 60)
(67804, 60, 1)
(2988, 60, 1)


In [7]:
# Check sample train and test features
print(x_train[0])
print(x_test[7])

[[ 0.012]
 [-0.055]
 [-0.065]
 [ 0.046]
 [-0.082]
 [-0.106]
 [ 0.075]
 [-0.069]
 [-0.143]
 [ 0.106]
 [-0.031]
 [-0.176]
 [ 0.101]
 [-0.03 ]
 [-0.084]
 [ 0.098]
 [ 0.002]
 [-0.139]
 [ 0.039]
 [ 0.028]
 [-0.174]
 [-0.021]
 [ 0.038]
 [-0.189]
 [ 0.083]
 [ 0.027]
 [-0.073]
 [ 0.072]
 [ 0.056]
 [-0.136]
 [-0.003]
 [ 0.074]
 [-0.165]
 [-0.07 ]
 [ 0.072]
 [-0.175]
 [ 0.065]
 [ 0.076]
 [-0.067]
 [ 0.051]
 [ 0.1  ]
 [-0.135]
 [-0.021]
 [ 0.111]
 [-0.146]
 [-0.082]
 [ 0.102]
 [-0.138]
 [ 0.046]
 [ 0.111]
 [-0.064]
 [ 0.048]
 [ 0.14 ]
 [-0.112]
 [ 0.002]
 [ 0.15 ]
 [-0.113]
 [-0.044]
 [ 0.142]
 [-0.104]]
[[-0.012]
 [-0.039]
 [-0.124]
 [ 0.028]
 [-0.076]
 [-0.196]
 [ 0.092]
 [-0.065]
 [-0.248]
 [ 0.163]
 [-0.073]
 [-0.292]
 [ 0.205]
 [-0.165]
 [-0.149]
 [ 0.258]
 [-0.12 ]
 [-0.255]
 [ 0.244]
 [-0.05 ]
 [-0.328]
 [ 0.215]
 [ 0.007]
 [-0.365]
 [ 0.245]
 [-0.094]
 [-0.119]
 [ 0.312]
 [ 0.04 ]
 [ 0.693]
 [ 0.249]
 [ 0.038]
 [-0.282]
 [ 0.187]
 [ 0.067]
 [-0.295]
 [ 0.255]
 [-0.019]
 [-0.101]
 [ 0.287]

In [8]:
# Number of classes according standard Indonesian Language Alphabets
num_classes = 5

# Using the Keras.Utils to put the label categorically 
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [9]:
# One Dimensional Convolutional Neural Network model, Train will be feed to 1 Dimension Convolutional Neural Network
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu", input_shape=x_train.shape[1:3]),
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(rate=0.2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(num_classes, activation='softmax')])

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 60, 32)            192       
                                                                 
 conv1d_1 (Conv1D)           (None, 60, 32)            5152      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 32)           0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 30, 64)            10304     
                                                                 
 conv1d_3 (Conv1D)           (None, 30, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 15, 64)           0         
 1D)                                                    

In [10]:
#Train the Model
model.fit(x_train, y_train, epochs=40, batch_size=20, validation_data=(x_test, y_test))

Epoch 1/40
3391/3391 [==============================] - 260s 73ms/step - loss: 0.4980 - accuracy: 0.8029 - val_loss: 4.6728 - val_accuracy: 0.0499
Epoch 2/40
3391/3391 [==============================] - 257s 76ms/step - loss: 0.2132 - accuracy: 0.9265 - val_loss: 4.6686 - val_accuracy: 0.0636
Epoch 3/40
3391/3391 [==============================] - 257s 76ms/step - loss: 0.1633 - accuracy: 0.9439 - val_loss: 5.8625 - val_accuracy: 0.0281
Epoch 4/40
3391/3391 [==============================] - 257s 76ms/step - loss: 0.1380 - accuracy: 0.9538 - val_loss: 6.2342 - val_accuracy: 0.0184
Epoch 5/40
3391/3391 [==============================] - 257s 76ms/step - loss: 0.1205 - accuracy: 0.9603 - val_loss: 7.4632 - val_accuracy: 0.0408
Epoch 6/40
3391/3391 [==============================] - 244s 72ms/step - loss: 0.1077 - accuracy: 0.9639 - val_loss: 6.0267 - val_accuracy: 0.0254
Epoch 7/40
3391/3391 [==============================] - 259s 76ms/step - loss: 0.1020 - accuracy: 0.9672 - val_loss: 7

In [11]:
#Saving the model into H5 system file
save_model = "CNN_origin.h5"
model.save(save_model)
print("Model Saved into", save_model)

Model Saved into CNN_origin.h5
